# Analysis of cold start problem in session-based recommendation
A comparison of sesison item knn vs adding deep reinforcement learning to session-based recommendation to solve the cold start problem.

In [ ]:
from datapreprocessing import DataPreprocessor

